<a href="https://colab.research.google.com/github/brianr482/optimized-sudoku-solver/blob/master/sudoku_bt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrantes:

* Valeria Álvarez
* Oskhar Arrieta
* Brian Ramírez

El resultado se encuentra en el archivo generado al final (logs.txt)

Instrucciones
1. Se montó una unidad en drive para cargar los archivos y generar el archivo salida. Por lo tanto, se deberá mofificar la variable DRIVE_ROOT_PATH, para que apunte al path donde estarán los archivos cargados.
2. Los archivos de prueba se encuentran alojados en la carpeta "test_files", estos se deben mover a la ruta montada en drive en el paso 1. 
3. Se debe correr cada bloque en el orden el que está para que funcione correctamente.

In [0]:
# Import libs
import numpy as np
import math
from google.colab import drive
drive.mount('/content/drive')
DRIVE_ROOT_PATH = '/content/drive/Shared drives/AI/Sudoku Backtracking/'

In [0]:
# Read the text file and parse it to a np matrix
file = open(f'{DRIVE_ROOT_PATH}unsolvable.txt')
parsed_array = []
for line in file:
  split_line = line.rstrip('\n').split(',')
  parsed_array += [[int(val) if val != '*' else None for val in split_line]]
sudoku_state = np.array(parsed_array)
BOARD_SIZE = len(sudoku_state[0])

In [0]:
# Define finishing constraint checker
def finishing_checker(sst):
  return not None in sst

In [0]:
# Define the constraints checker function
def constraint_checker(current_sst, value, i, j):
  # Validate the row constraint
  if value in current_sst[i]:
    return False

  # Validate the column constraint
  if value in current_sst[:, j]:
    return False
  
  # Validate the block constraint
  block_i = i // 3
  block_j = j // 3
  block = current_sst[block_i * 3 : block_i * 3 + 3, block_j * 3 : block_j * 3 + 3]
  if value in block:
    return False

  return True

In [0]:
# Generate domain matrix
def generate_domains(sst):
  domain = np.empty([BOARD_SIZE, BOARD_SIZE], dtype=list)
  for i in range(BOARD_SIZE):
    for j in range(BOARD_SIZE):
      cell_domain = None
      if sst[i,j] == None:
        cell_domain = list([])
        for number in range(1, 10):
          if (constraint_checker(sst, number, i, j)):
            cell_domain += [number]
      domain[i, j] = cell_domain
  return domain

In [0]:
def get_most_restricted(domain_matrix):
  max = math.inf
  coord = list([None, None])
  for i in range(BOARD_SIZE):
    for j in range(BOARD_SIZE):
      domain = domain_matrix[i, j]
      # print(domain)
      if domain != None and len(domain) < max:
        max = len(domain)
        coord[0] = i
        coord[1] = j
  return coord if max != math.inf else None

In [0]:
def affected_cells(current_sst, i, j, num, domain_matrix):
  affected = 0
  visited = list([])
  # Check the affected row
  row = current_sst[i]
  for pos in range(len(row)):
    val = row[pos]
    if val == None and num in domain_matrix[i, pos]:
      affected += 1
      visited += [[i, pos]]

  # Check the affected column
  column = current_sst[:, j]
  for pos in range(len(column)):
    val = column[pos]
    if val == None and num in domain_matrix[pos, j]:
      affected += 1
      visited += [[pos, j]]
  
  # Check the affected block
  block_i = i // 3  
  block_j = j // 3
  block = current_sst[block_i * 3 : block_i * 3 + 3, block_j * 3 : block_j * 3 + 3]
  for r in range(len(block[0])):
    for col in range(len(block[0])):
      val = block[r, col]
      real_i = r + block_i * 3
      real_j = col + block_j * 3
      if val == None and not [real_i, real_j] in visited and num in domain_matrix[real_i, real_j]:
        affected += 1

  return affected

In [0]:
# Find the least restrictive value
def get_least_restrictive_value(sst, i, j, options, domain_matrix):
  min_affected_cells = math.inf
  number = None
  for num in options:
    aff = affected_cells(sst, i, j, num, domain_matrix)
    if aff < min_affected_cells:
      min_affected_cells = aff
      number = num
  return number

In [0]:
# Format the sudoku state in order to show it on the log file
def display_sudoku_state(sst):
  sudoku = list([])
  for i in range(BOARD_SIZE):
    sudoku += [[]]
    for j in range(BOARD_SIZE):
      val = sst[i, j]
      sudoku[i] += [val if val != None else '*']
  return np.array(sudoku)

In [0]:
# Define backtracking function
def solver(sst, domain_matrix, iterations):
  most_restricted = get_most_restricted(domain_matrix)
  if most_restricted != None:
    i = most_restricted[0]
    j = most_restricted[1]
    options = list(domain_matrix[i, j])
    while len(options) > 0:
      least_restrictive_value = get_least_restrictive_value(sst, i, j, options, domain_matrix)
      new_sst = sst.copy()
      new_sst[i, j] = least_restrictive_value
      options.remove(least_restrictive_value)
      log_file.write(f'Iteration: {iterations} \n')
      log_file.write(f'Most restricted: {most_restricted} \n')
      log_file.write(f'Least restrictive: {least_restrictive_value} \n')
      log_file.write('State: \n')
      log_file.write(np.array2string(display_sudoku_state(new_sst)))
      log_file.write('\n')
      log_file.write('\n')
      res = solver(new_sst, generate_domains(new_sst), iterations + 1)

      # Find a solution, return it
      if type(res) is np.ndarray:
        return res
  elif finishing_checker(sst):
    return sst

In [0]:
log_file = open(f'{DRIVE_ROOT_PATH}logs.txt', 'w')
sst = np.array(sudoku_state)
domain_matrix = generate_domains(sst)
res = solver(sst, domain_matrix, 0)
if not type(res) is np.ndarray:
  log_file.write('No solution')
log_file.close()